In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Basic RAG with Weaviate

Now - let's try performing RAG with the chunks that we've created. 

We will:
- Load & chunk a document
- Add the chunks to Weaviate, and generate vectors
- And perform RAG

We assume some familiarity with Weaviate here. 

(If not, check out the [Weaviate Quickstart](https://docs.weaviate.io/weaviate/quickstart), or ask questions in the live session!)

### Load and chunk a document

In [ ]:
from pathlib import Path

def get_chunks_using_markers(src_text: str) -> list[str]:
    """
    Split the source text into chunks using markers.
    """
    marker = "\n##"

    # Split by marker and reconstruct with markers (except first chunk)
    parts = src_text.split(marker)
    chunks = []

    # Add first chunk if it exists and isn't empty
    if parts[0].strip():
        chunks.append(parts[0].strip())

    # Add remaining chunks with markers reattached
    for part in parts[1:]:
        if part.strip():
            chunks.append(marker + part.strip())

    return chunks


md_file = Path("data/parsed/hai_ai_index_report_2025_chapter_2-parsed-text.md")
md_text = md_file.read_text(encoding="utf-8")
chunk_texts = get_chunks_using_markers(md_text)

### Set up Weaviate

In [ ]:
import weaviate
import os

# Use Embedded Weaviate with:
# version: latest available (e.g. "1.32.0"), headers = {"X-Cohere-Api-Key": os.getenv("COHERE_API_KEY")}, env vars: {"LOG_LEVEL": "error"}
# ADD YOUR CODE HERE

### Set up a collection

In [ ]:
client.collections.delete("Chunks")

In [ ]:
from weaviate.classes.config import Property, DataType, Configure, Tokenization

client.collections.create(
    name="Chunks",
    properties=[
        Property(
            name="document_title",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk_number",
            data_type=DataType.INT,
        ),
        Property(
            name="filename",
            data_type=DataType.TEXT,
            tokenization=Tokenization.FIELD
        ),
    ],
    vector_config=[
        # Add `Configure.Vectors.text2vec_cohere` vector to the collection with:
        # name: "default", source properties: ["document_title", "chunk"], and model: "embed-v4.0"
        # ADD YOUR CODE HERE
    ],
    generative_config=Configure.Generative.cohere(
        model="command-r"
    )
)

### Import data

In [ ]:
chunks = client.collections.get("Chunks")

In [ ]:
from tqdm import tqdm

with chunks.batch.fixed_size(batch_size=100) as batch:
    for i, chunk_text in tqdm(enumerate(chunk_texts)):
        obj = {
            "document_title": "Stanford HAI Report 2025",
            "filename": "data/pdfs/hai_ai_index_report_2025_chapter_2.pdf",
            "chunk": chunk_text,
            "chunk_number": i + 1,
        }

        # Add object to batch for import with (batch.add_object())
        # ADD YOUR CODE HERE

### RAG queries



In [ ]:
# Try a RAG query with:
# query (what to search for): "how to clean the washing machine" and
# grouped_task (prompt): "Briefly, what tasks do I need to perform to regularly maintain and clean the washing machine?"
# limit (how many objects to fetch): 10
# ADD YOUR CODE HERE

print("Query response:")
print(response.generative.text)

### Recap - what's happening under the hood

![assets/llm_3_rag_weaviate.png](assets/llm_3_rag_weaviate.png)

We can review the passages:

In [ ]:
print("Supporting passages:")
for o in response.objects:
    print(f"\n> Object: {o.uuid}:")
    print(o.properties['chunk'][:200]+"...")

In [ ]:
client.close()